# Corpora and vector spaces
https://radimrehurek.com/gensim/tut1.html

In [21]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print 'Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER)

from gensim import corpora, models, similarities

Folder "/var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T" will be used to save temporary dictionary and corpus.


In [22]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [23]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

# lemmatization - get original form of words ????

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [[token for token in text if frequency[token] > 1] for text in texts]
from pprint import pprint
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [24]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
print dictionary

2018-03-26 18:11:34,690 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-26 18:11:34,692 : INFO : built Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...) from 9 documents (total 29 corpus positions)
2018-03-26 18:11:34,693 : INFO : saving Dictionary object under /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.dict, separately None
2018-03-26 18:11:34,694 : INFO : saved /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.dict


Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


In [25]:
item_document_fre = dictionary.dfs
for k in sorted(dictionary.keys()):
    print k, dictionary[k], item_document_fre[k]

0 interface 2
1 computer 2
2 human 2
3 response 2
4 time 2
5 survey 2
6 system 3
7 user 3
8 eps 2
9 trees 3
10 graph 3
11 minors 2


In [26]:
print dictionary.token2id

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [27]:
# bag of words
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)
corpus

2018-03-26 18:11:49,311 : INFO : storing corpus in Matrix Market format to /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm
2018-03-26 18:11:49,312 : INFO : saving sparse matrix to /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm
2018-03-26 18:11:49,313 : INFO : PROGRESS: saving document #0
2018-03-26 18:11:49,314 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2018-03-26 18:11:49,315 : INFO : saving MmCorpus index to /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]

In [28]:
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))

2018-03-26 18:11:53,534 : INFO : loaded corpus index from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm.index
2018-03-26 18:11:53,536 : INFO : initializing corpus reader from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm
2018-03-26 18:11:53,537 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


# Topics and transformations
https://radimrehurek.com/gensim/tut2.html

In [29]:
tfidf = models.TfidfModel(corpus)

2018-03-26 18:11:56,549 : INFO : collecting document frequencies
2018-03-26 18:11:56,550 : INFO : PROGRESS: processing document #0
2018-03-26 18:11:56,552 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [30]:
corpus_tfidf = tfidf[corpus]
for i in corpus_tfidf:
    print i

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [31]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2018-03-26 18:12:22,182 : INFO : using serial LSI version on this node
2018-03-26 18:12:22,184 : INFO : updating model with new documents
2018-03-26 18:12:22,185 : INFO : preparing a new chunk of documents
2018-03-26 18:12:22,187 : INFO : using 100 extra samples and 2 power iterations
2018-03-26 18:12:22,188 : INFO : 1st phase: constructing (12, 102) action matrix
2018-03-26 18:12:22,191 : INFO : orthonormalizing (12, 102) action matrix
2018-03-26 18:12:22,196 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-03-26 18:12:22,198 : INFO : computing the final decomposition
2018-03-26 18:12:22,199 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2018-03-26 18:12:22,201 : INFO : processed documents up to #9
2018-03-26 18:12:22,203 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-03-26 18:12:22,203 : INFO : topic #

In [32]:
lsi.print_topics(-1)

2018-03-26 18:12:23,310 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-03-26 18:12:23,311 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  u'-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"response" + -0.320*"time" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [33]:
for doc in corpus_lsi:
    print doc

[(0, 0.066007833960906925), (1, -0.52007033063618413)]
[(0, 0.19667592859142993), (1, -0.7609563167700033)]
[(0, 0.089926399724468795), (1, -0.72418606267525032)]
[(0, 0.075858476521785526), (1, -0.63205515860034245)]
[(0, 0.1015029918498053), (1, -0.57373084830029464)]
[(0, 0.70321089393782965), (1, 0.16115180214026226)]
[(0, 0.87747876731198149), (1, 0.16758906864659945)]
[(0, 0.90986246868185638), (1, 0.14086553628719553)]
[(0, 0.61658253505692795), (1, -0.053929075663890172)]


In [34]:
# lsi.save('/tmp/model.lsi') # same for tfidf, lda, ...
# lsi = models.LsiModel.load('/tmp/model.lsi')

In [35]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = lda[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2018-03-26 18:12:25,651 : INFO : using symmetric alpha at 0.5
2018-03-26 18:12:25,653 : INFO : using symmetric eta at 0.0833333333333
2018-03-26 18:12:25,654 : INFO : using serial LDA version on this node
2018-03-26 18:12:25,656 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2018-03-26 18:12:25,657 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-03-26 18:12:25,668 : INFO : -3.296 per-word bound, 9.8 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2018-03-26 18:12:25,669 : INFO : PROGRESS: pass 0, at document #9/9
2018-03-26 18:12:25,680 : INFO : topic #0 (0.500): 0.180*"graph" + 0.175*"trees" + 0.126*"minors" + 0.081*"survey" + 0.078*"interface" + 0.069*"human" + 0.068*"com

In [36]:
lda.print_topics(-1)

2018-03-26 18:12:26,078 : INFO : topic #0 (0.500): 0.180*"graph" + 0.175*"trees" + 0.126*"minors" + 0.081*"survey" + 0.078*"interface" + 0.069*"human" + 0.068*"computer" + 0.048*"system" + 0.048*"user" + 0.046*"time"
2018-03-26 18:12:26,080 : INFO : topic #1 (0.500): 0.176*"system" + 0.134*"user" + 0.099*"eps" + 0.094*"response" + 0.093*"time" + 0.077*"computer" + 0.076*"human" + 0.070*"interface" + 0.068*"survey" + 0.041*"trees"


[(0,
  u'0.180*"graph" + 0.175*"trees" + 0.126*"minors" + 0.081*"survey" + 0.078*"interface" + 0.069*"human" + 0.068*"computer" + 0.048*"system" + 0.048*"user" + 0.046*"time"'),
 (1,
  u'0.176*"system" + 0.134*"user" + 0.099*"eps" + 0.094*"response" + 0.093*"time" + 0.077*"computer" + 0.076*"human" + 0.070*"interface" + 0.068*"survey" + 0.041*"trees"')]

In [37]:
for doc in corpus_lda:
    print doc

[(0, 0.22032041379040368), (1, 0.77967958620959643)]
[(0, 0.085746661922303624), (1, 0.91425333807769638)]
[(0, 0.11522074285478598), (1, 0.8847792571452141)]
[(0, 0.111350669799903), (1, 0.88864933020009695)]
[(0, 0.13635231232623984), (1, 0.8636476876737601)]
[(0, 0.73836527342640101), (1, 0.2616347265735991)]
[(0, 0.82612516231795508), (1, 0.17387483768204492)]
[(0, 0.86918652293694976), (1, 0.1308134770630503)]
[(0, 0.85646237685566151), (1, 0.14353762314433854)]


# Similarity
https://radimrehurek.com/gensim/tut3.html

In [38]:
dictionary = corpora.Dictionary.load(os.path.join(TEMP_FOLDER, 'deerwester.dict'))
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'deerwester.mm'))

2018-03-26 18:12:29,294 : INFO : loading Dictionary object from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.dict
2018-03-26 18:12:29,295 : INFO : loaded /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.dict
2018-03-26 18:12:29,297 : INFO : loaded corpus index from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm.index
2018-03-26 18:12:29,298 : INFO : initializing corpus reader from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.mm
2018-03-26 18:12:29,299 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


## Use lsi vector

In [52]:
lsi = models.LsiModel(corpus, id2word=dictionary)#, num_topics=2)

2018-03-26 18:17:46,699 : INFO : using serial LSI version on this node
2018-03-26 18:17:46,700 : INFO : updating model with new documents
2018-03-26 18:17:46,702 : INFO : preparing a new chunk of documents
2018-03-26 18:17:46,703 : INFO : using 100 extra samples and 2 power iterations
2018-03-26 18:17:46,703 : INFO : 1st phase: constructing (12, 300) action matrix
2018-03-26 18:17:46,704 : INFO : orthonormalizing (12, 300) action matrix
2018-03-26 18:17:46,706 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-03-26 18:17:46,707 : INFO : computing the final decomposition
2018-03-26 18:17:46,707 : INFO : keeping 9 factors (discarding 0.000% of energy spectrum)
2018-03-26 18:17:46,708 : INFO : processed documents up to #9
2018-03-26 18:17:46,709 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"response" + 0.265*"time" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2018-03-26 18:17:46,710 : INFO : topic #1(2.

In [53]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print vec_lsi, vec_bow

[(0, 0.46182100453271624), (1, -0.070027665278999451), (2, -0.12452907551899037), (3, 1.009712558443856), (4, -0.21303040605626575), (5, -0.59593845338206686), (6, -0.22041753546094373), (7, -0.0018778773554748163), (8, 0.085766854949955729)] [(1, 1), (2, 1)]


In [54]:
index = similarities.MatrixSimilarity(lsi[corpus])

2018-03-26 18:17:57,100 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-03-26 18:17:57,102 : INFO : creating matrix with 9 documents and 9 features


In [55]:
index.save(os.path.join(TEMP_FOLDER, 'deerwester.index'))
index = similarities.MatrixSimilarity.load(os.path.join(TEMP_FOLDER, 'deerwester.index'))

2018-03-26 18:17:57,595 : INFO : saving MatrixSimilarity object under /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.index, separately None
2018-03-26 18:17:57,597 : INFO : saved /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.index
2018-03-26 18:17:57,598 : INFO : loading MatrixSimilarity object from /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.index
2018-03-26 18:17:57,599 : INFO : loaded /var/folders/cm/470833qd7lg343v6z69fh9gm0000gn/T/deerwester.index


In [56]:
sims = index[vec_lsi]
print list(enumerate(sims))

[(0, 0.88310492), (1, 0.31222469), (2, -1.4062342e-08), (3, 0.31222472), (4, -4.3908033e-09), (5, -6.4911487e-10), (6, -8.2508014e-11), (7, -3.9696437e-09), (8, -3.3845107e-09)]


## Use TFIDF vector

In [46]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_tfidf = tfidf[vec_bow]
print vec_tfidf, vec_bow

[(1, 0.7071067811865476), (2, 0.7071067811865476)] [(1, 1), (2, 1)]


In [48]:
index_tfidf = similarities.MatrixSimilarity(tfidf[corpus])
sims = index_tfidf[vec_tfidf]
print list(enumerate(sims))

2018-03-26 18:14:26,283 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-03-26 18:14:26,285 : INFO : creating matrix with 9 documents and 12 features


[(0, 0.81649655), (1, 0.31412902), (2, 0.0), (3, 0.34777319), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


In [45]:
texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [51]:
list(lsi[corpus])

[[(0, 0.65946640597973993), (1, 0.14211544403729853)],
 [(0, 2.0245430433828755), (1, -0.42088758246302532)],
 [(0, 1.5465535813286546), (1, 0.3235891942571214)],
 [(0, 1.8111412473028834), (1, 0.58905249699325146)],
 [(0, 0.93367380356343466), (1, -0.27138940499375541)],
 [(0, 0.012746183038294777), (1, -0.49016179245310371)],
 [(0, 0.048882032060470926), (1, -1.1129470269929536)],
 [(0, 0.080638360994106983), (1, -1.5634559463442637)],
 [(0, 0.27381003921275737), (1, -1.3469415849537685)]]